In [1]:
#!/usr/bin/env python

import argparse
import matplotlib.pyplot as plt
import numpy as np
import json
import csv
import pandas as pd
import pg8000
import os

def json_feature_parse(file_name):
    with open(file_name, "r") as f:
        data = json.load(f)
    path = data["specimens"][0]["cell_ephys_features"]["long_squares"]
    specimen_id = data['id']
    tau = path["tau"]
    sag = path['sag']
    ri = path["input_resistance"]
    sag_amp =path['v_baseline'] - (path['vm_for_sag']*sag)
    ISI_mean = path['hero_sweep']['mean_isi']
    adaptation_index = path['hero_sweep']['adapt']
    capacitance = tau/ri*1000
    AP_LS_amp = path['hero_sweep']['spikes'][0]['peak_v'] - path['hero_sweep']['spikes'][0]['threshold_v']
    AP_SS_amp = data["specimens"][0]["cell_ephys_features"]["short_squares"]['mean_spike_0']['peak_v'] - data["specimens"][0]["cell_ephys_features"]["short_squares"]['mean_spike_0']['threshold_v']
    upstroke_downstroke_ratio = path['rheobase_sweep']['spikes'][0]['upstroke_downstroke_ratio']
    v_rest = path['v_baseline']
    rheobase_sweep = path['rheobase_sweep']['id']
    ss_latency= data["specimens"][0]["cell_ephys_features"]['short_squares']['common_amp_sweeps'][0]['latency']
    ls_latency = path['hero_sweep']['latency']
    instant_sweep = data["specimens"][0]["cell_ephys_features"]['short_squares']['common_amp_sweeps'][0]['id']
    rheobase_i = path['rheobase_i']
    hero_rate = path['hero_sweep']['avg_rate']
    try:
        human_df['noise_1'].iloc[i] = sweep_numbers['Noise 1'][0]
        human_df['noise_2'].iloc[i] = sweep_numbers['Noise 2'][0]
    except IndexError:
        pass
    human_df['hero_rate'].iloc[i] = cell_features['long_squares']['hero_sweep']['avg_rate']
    spikes = cell_features['long_squares']['hero_sweep']['spikes']
    if len(spikes) > 1:
        human_df['fast_trough'].iloc[i] = np.mean([spikes[f]['fast_trough_v'] for f in range(len(spikes))])
    else:
        human_df['fast_trough'].iloc[i] = cell_features['long_squares']['hero_sweep']['spikes'][0]['fast_trough_v']
    human_df['hero_sweep'].iloc[i] = cell_features['long_squares']['hero_sweep']['id']
    human_df['hero_ap_peak_t'].iloc[i] = cell_features['long_squares']['hero_sweep']['spikes'][0]['peak_t']
    human_df['hero_ap_threshold_v'].iloc[i] = cell_features['long_squares']['hero_sweep']['spikes'][0]['threshold_v']
    human_df['IH'].iloc[i] = {i['id']:i['stim_amp'] for i in cell_features['long_squares']['subthreshold_membrane_property_sweeps'] }
    input_resistance = data["specimens"][0]["cell_ephys_features"]["long_squares"]["input_resistance"]
    thresh_i = data["specimens"][0]["cell_ephys_features"]["long_squares"]["rheobase_sweep"]["stim_amp"]
    up_down_ratio = data["specimens"][0]["cell_ephys_features"]["long_squares"]["rheobase_sweep"]["spikes"][0]["upstroke_downstroke_ratio"]
    adapt = data["specimens"][0]["cell_ephys_features"]["long_squares"]["hero_sweep"]["adapt"]
    Firing_rate = data["specimens"][0]["cell_ephys_features"]["long_squares"]["hero_sweep"]["mean_isi"]

    features = []
    features.append(tau)
    features.append(input_resistance)
    features.append(thresh_i)
    features.append(up_down_ratio)
    features.append(adapt)
    features.append(Firing_rate)

    return features

def get_spec_name(specimen):
    conn = pg8000.connect(user="limsreader", host="limsdb2", database="lims2", password="limsro", port=5432)
    cur = conn.cursor()
    
    cur.execute("SELECT s.name, s.ephys_roi_result_id, s.id FROM specimens s WHERE s.name LIKE %s", ('%' + specimen,))
    result = cur.fetchone()
    if result is None:
        print "Could not find specimen result for " + specimen + ". Skipping..."
        return " "
    #print "Specimen: " + result[0]
    #print "EphysRoiResult: " + str(result[1])
    specimen_name = result[0]
    ephys_roi_result_id = result[1]


    cur.execute("SELECT f.filename, f.storage_directory FROM well_known_files f \
                 WHERE f.attachable_type = 'EphysRoiResult' AND f.attachable_id = %s AND f.filename LIKE '%%ephys_features.json'", 
                 (ephys_roi_result_id,))
    result = cur.fetchone()
    print result
    print 'hi'
    if result is None:
        print "Could not find json file for " + specimen + ". Skipping..."
        return None
    WinP = LinuxtoWindow(result[1])
    json_path = WinP + result[0]
    # print "json file: " + json_path

    return specimen_name, json_path
    conn.close()
    cur.close()

    
    
def LinuxtoWindow(Linuxpath):
    SplitLinux = Linuxpath.split('/')
    return (r"\\" + os.path.join('titan','cns', SplitLinux[2], SplitLinux[4],SplitLinux[5],''))

with open("C:/SpecList.csv", 'rb') as csvfile:
    listreader = csv.reader(csvfile, delimiter=' ', quotechar='|')
    namelist = []
    j_pathlist = []
    for row in listreader:
        name, j_path = get_spec_name(row[0])
        namelist.append(name)
        j_pathlist.append(j_path)

FeatSumm = ['Name', 'tau', 'input_resistance', 'thresh_i', 'up_down_ratio', 'adapt', 'FR']
print 'featsum'


for ix, name in enumerate(namelist):

    y = j_pathlist[ix]
    print y
    try:
        a = json_feature_parse(y)
        print a
        a.insert(0, name)
        FeatSumm = np.vstack([FeatSumm,a])
    except:
        print name + " has an error"


myfile = open('c:/Summary.csv', 'wb')
wr = csv.writer(myfile, quoting=csv.QUOTE_ALL)
for values in FeatSumm:
    wr.writerow(values)
    


[u'486748227_ephys_features.json', u'/projects/humancelltypes/vol1/prod4/Ephys_Roi_Result_486748227/']
hi
featsum
\\titan\cns\humancelltypes\prod4\Ephys_Roi_Result_486748227\486748227_ephys_features.json
H15.03.005.01.14.01 has an error


In [2]:
myfile.close()

In [3]:
json_feature_parse(y)

NameError: global name 'sweep_numbers' is not defined

In [4]:
with open(y, "r") as f:
    data = json.load(f)

In [12]:
len(data["specimens"][0]["cell_ephys_features"]["long_squares"]['hero_sweep']['spikes'])

3

In [14]:
data['id']
#watch the sweep numbers
data['specimens'][0]['ephys_sweeps'][50]['ephys_stimulus']['ephys_stimulus_type']['name']

486748227

In [70]:
import argparse
import matplotlib.pyplot as plt
import numpy as np
import json
import csv
import pandas as pd
import pg8000
import os

def json_feature_parse(file_name):
    with open(file_name, "r") as f:
        data = json.load(f)

    tau = data["specimens"][0]["cell_ephys_features"]["long_squares"]["tau"]
    print "tau"
    input_resistance = data["specimens"][0]["cell_ephys_features"]["long_squares"]["input_resistance"]
    thresh_i = data["specimens"][0]["cell_ephys_features"]["long_squares"]["rheobase_sweep"]["stim_amp"]
    up_down_ratio = data["specimens"][0]["cell_ephys_features"]["long_squares"]["rheobase_sweep"]["spikes"][0]["upstroke_downstroke_ratio"]
    adapt = data["specimens"][0]["cell_ephys_features"]["long_squares"]["hero_sweep"]["adapt"]
    Firing_rate = data["specimens"][0]["cell_ephys_features"]["long_squares"]["hero_sweep"]["mean_isi"]

    features = []
    features.append(tau)
    features.append(input_resistance)
    features.append(thresh_i)
    features.append(up_down_ratio)
    features.append(adapt)
    features.append(Firing_rate)

    return features

def get_spec_name(ephys_roi_result_id,):
    conn = pg8000.connect(user="limsreader", host="limsdb2", database="lims2", password="limsro", port=5432)
    cur = conn.cursor()
    
    
    

    cur.execute("SELECT f.filename, f.storage_directory FROM well_known_files f \
                 WHERE f.attachable_type = 'EphysRoiResult' AND f.attachable_id = %s AND f.filename LIKE '%%ephys_features.json'", 
                 (ephys_roi_result_id,))
    result = cur.fetchone()
    print result
    print 'hi'
    if result is None:
        print "Could not find json file for " + ephys_roi_result_id + ". Skipping..."
        return None
    WinP = LinuxtoWindow(result[1])
    json_path = WinP + result[0]
    # print "json file: " + json_path

    return specimen_name, json_path
    conn.close()
    cur.close()

    
    
def LinuxtoWindow(Linuxpath):
    SplitLinux = Linuxpath.split('/')
    return (r"\\" + os.path.join('titan','cns', SplitLinux[2], SplitLinux[4],SplitLinux[5],''))

with open("C:/SpecList.csv", 'rb') as csvfile:
    listreader = csv.reader(csvfile, delimiter=' ', quotechar='|')
    namelist = []
    j_pathlist = []
    for row in listreader:
        name, j_path = get_spec_name(row[0])
        namelist.append(name)
        j_pathlist.append(j_path)

FeatSumm = ['Name', 'tau', 'input_resistance', 'thresh_i', 'up_down_ratio', 'adapt', 'FR']
print 'featsum'


for ix, name in enumerate(namelist):

    y = j_pathlist[ix]
    print y
    try:
        a = json_feature_parse(y)
        print a
        a.insert(0, name)
        #FeatSumm = np.vstack([FeatSumm,a])
    except:
        print name + " has an error"


test_df = pd.Series(data = FeatSumm[1])

#myfile.close()

ProgrammingError: (u'ERROR', u'22P02', u'invalid input syntax for integer: "H15.03.005.01.14.01"', u'numutils.c', u'62', u'pg_atoi', u'', u'')

In [92]:
def get_spec_name(specimen):
    conn = pg8000.connect(user="limsreader", host="limsdb2", database="lims2", password="limsro", port=5432)
    cur = conn.cursor()
    
    cur.execute("SELECT s.name, s.ephys_roi_result_id, s.id FROM specimens s WHERE s.name LIKE %s", ('%' + specimen,))
    result = cur.fetchone()
    if result is None:
        print "Could not find specimen result for " + specimen + ". Skipping..."
        return " "
    #print "Specimen: " + result[0]
    #print "EphysRoiResult: " + str(result[1])
    specimen_name = result[0]
    ephys_roi_result_id = result[1]
    print result

    cur.execute("SELECT f.filename, f.storage_directory FROM well_known_files f \
                 WHERE f.attachable_type = 'EphysRoiResult' AND f.attachable_id = %s AND f.filename LIKE '%%ephys_features.json'", 
                 (ephys_roi_result_id,))
    result = cur.fetchone()
    print result
    print 'hi'
    if result is None:
        print "Could not find json file for " + specimen + ". Skipping..."
        return None
    WinP = LinuxtoWindow(result[1])
    json_path = WinP + result[0]
    # print "json file: " + json_path

    return specimen_name, json_path
    conn.close()
    cur.close()

In [93]:
get_spec_name('H16.06.009.01.01.06.01')

[u'H16.06.009.01.01.06.01', 528602937, 528602977]
[u'528602937_ephys_features.json', u'/projects/mousecelltypes/vol1/prod799/Ephys_Roi_Result_528602937/']
hi


(u'H16.06.009.01.01.06.01',
 u'\\\\titan\\cns\\mousecelltypes\\prod799\\Ephys_Roi_Result_528602937\\528602937_ephys_features.json')

In [45]:
from allensdk.core.cell_types_cache import CellTypesCache
from allensdk.core.cell_types_cache import ReporterStatus as RS
from allensdk.ephys.feature_extractor import EphysFeatureExtractor


ctc = CellTypesCache()
ephys_features = ctc.get_ephys_features()
cells = ctc.get_cells()
cell_index = {c['id']: c for c in cells}


v1_df = pd.DataFrame(data=ephys_features)
v1_df['cre_line'], v1_df['cre_reporter'],v1_df['location'] = None, None, None
v1_df['dendrite_type'],v1_df['depth'] = None, None
for i, spec_id in enumerate(v1_df['specimen_id']):
    v1_df['cre_line'][i] = cell_index[spec_id]['transgenic_line']
    v1_df['cre_reporter'][i] = cell_index[spec_id]['cell_reporter']['name']
    v1_df['dendrite_type'][i] = cell_index[spec_id]['dendrite_type']
    v1_df['depth'][i] = cell_index[spec_id]['cell_soma_locations'][0]['normalized_depth']


In [39]:
test_df = pd.DataFrame(data = a, columns = FeatSumm,index = (7,1))

ValueError: Shape of passed values is (1, 7), indices imply (7, 2)

In [26]:
test_df.head()

0    H15.03.005.01.14.01
1        0.0274351781901
2          151.718750596
3                  110.0
4          3.39776722464
dtype: object